In [1]:
import pickle as pkl

import lxml
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.datasets import imdb


from tqdm.notebook import tqdm

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Conv1D, Flatten, MaxPooling1D,\
                        AveragePooling1D, Concatenate, LeakyReLU, Embedding,\
                        GlobalMaxPooling1D,GlobalAveragePooling1D,GaussianNoise,BatchNormalization,Add
from tensorflow.keras.initializers import glorot_uniform
#from tensorflow.keras.utils.vis_utils import plot_model

import importlib
importlib.reload(tensorflow)
import deepchem as dc

In [ ]:
model = dc.models.GraphConvModel(n_tasks=1,mode='classification',dropout=0.2)

In [2]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    NOTE: Taken from sklearn example
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = [0,1]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    
    #ax.figsize((10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [3]:
import sys
from tqdm import tqdm
tqdm.pandas()

In [4]:
from ast import literal_eval


In [5]:
drugbank = pd.read_csv('data/intermediate/interactions_preprocessed.csv')

In [ ]:
drugbank['drug_fingerprint_encoded_padded']=drugbank['drug_fingerprint_encoded_padded'].progress_apply(lambda x: x.strip("[]").replace("'","").split(", "))

 25%|██▌       | 21442/84432 [00:33<01:29, 706.14it/s]

In [ ]:
drugbank['target_gene_encoded_padded']=drugbank['target_gene_encoded_padded'].progress_apply(lambda x: x.strip("[]").replace("'","").split(", "))

In [ ]:
train_df, test_df = train_test_split(drugbank, test_size =.30)

In [ ]:
val_df, test_df = train_test_split(test_df, test_size = .50)

In [ ]:
train_df["Y"].value_counts()

In [ ]:
val_df["Y"].value_counts()

In [ ]:
test_df["Y"].value_counts()

In [ ]:
%%time
X_0_train = np.array(list(train_df['target_gene_encoded_padded'])).astype(int) + 1
X_1_train = np.array(list(train_df['drug_fingerprint_encoded_padded'])).astype(int) + 1
Y_train   = np.array(list(train_df['Y'])).astype(int)

In [ ]:
%%time
X_0_val = np.array(list(val_df['target_gene_encoded_padded'])).astype(int) + 1
X_1_val = np.array(list(val_df['drug_fingerprint_encoded_padded'])).astype(int) + 1
Y_val   = np.array(list(val_df['Y'])).astype(int)

In [ ]:
%%time
X_0_test = np.array(list(test_df['target_gene_encoded_padded'])).astype(int) + 1
X_1_test = np.array(list(test_df['drug_fingerprint_encoded_padded'])).astype(int) + 1
Y_test   = np.array(list(test_df['Y'])).astype(int)